In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_row', 50)

train = pd.read_csv('../data/train_ratings.csv')
test = pd.read_csv('../data/test_ratings.csv')
books = pd.read_csv('../data/books.csv')
users = pd.read_csv('../data/users.csv')

n = 10 # N개 이하 범주는 Others로, N은 유동적으로 할 것

In [2]:
# books 테이블 전처리 부분 입니다.
# books의 이미지 변수를 지워줍니다.
# 제목과 요약 내용 변수를 지웁니다. 
# (이 변수들은 추후 사용 가능할수도 있으나 일단 지웁니다.)
# books의 publisher 변수 중 이름이 비슷한 변수들을 찾아 하나로 통일해줍니다.
books.drop(['book_title', 'summary', 'img_url', 'img_path'], axis = 1, inplace = True)

books_publishers = books.groupby('publisher')['isbn'].count().sort_values(ascending=False)
for i in books_publishers[books_publishers > 20].index: # 20 말고 10으로 하면 오류가 남..
    books['publisher'][books['publisher'].str.contains(i)] = i

In [3]:
# books의 카테고리 부분.

# 대괄호 써있는 카테고리 전치리
books.loc[books[books['category'].notnull()].index, 'category'] = books[books['category'].notnull()]['category'].apply(lambda x: re.sub('[\W_]+',' ',x).strip())
# 모두 소문자로 통일
books['category'] = books['category'].str.lower()

# 수작업으로 higt 카테고리로 통합
categories = ['garden','crafts','physics','adventure','music','fiction','nonfiction','science','science fiction','social','homicide',
 'sociology','disease','religion','christian','philosophy','psycholog','mathemat','agricult','environmental',
 'business','poetry','drama','literary','travel','motion picture','children','cook','literature','electronic',
 'humor','animal','bird','photograph','computer','house','ecology','family','architect','camp','criminal','language','india']

books['category_high'] = books['category'].copy()
for category in categories:
    books.loc[books[books['category'].str.contains(category,na=False)].index,'category_high'] = category

In [4]:
# language와 category_high NULL 값을 최빈값으로 채웁니다.
# 근거 : language == en일 때, category_high == fiction 일 때와
# 근거 : 값이 NULL 일 때 rating 평균이 7.0x로 유사한 형태.
books['language'].fillna('en', inplace = True)
books['category_high'].fillna('fiction', inplace = True)

In [5]:
# 출판연도 1970, 1980, 1990, 2000, 2020 으로 범주화 시킵니다.
# 딥러닝 과정에서 범주화 시키는 것이 유리합니다.
# 근거 : develop 파일에서 여러번 실험 결과 본 기준이 가장 rating을 잘 구분함.

books['years'] = books['year_of_publication'].copy()
books['years'][books['year_of_publication'] < 1970] = 1970
books['years'][(books['year_of_publication'] < 1980) * (books['year_of_publication'] >= 1970)] = 1980
books['years'][(books['year_of_publication'] < 1990) * (books['year_of_publication'] >= 1980)] = 1990
books['years'][(books['year_of_publication'] < 2000) * (books['year_of_publication'] >= 1990)] = 2000
books['years'][(books['year_of_publication'] >= 2000)] = 2020
books['years'] = books['years'].astype('int')
books['years'] = books['years'].astype('str')

In [6]:
# 사용한 변수를 제외하고 전처리가 잘 됬는지 확인합니다.
books.drop(['year_of_publication', 'category'], axis = 1, inplace = True)
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149570 entries, 0 to 149569
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   isbn           149570 non-null  object
 1   book_author    149570 non-null  object
 2   publisher      149570 non-null  object
 3   language       149570 non-null  object
 4   category_high  149570 non-null  object
 5   years          149570 non-null  object
dtypes: object(6)
memory usage: 6.8+ MB


In [7]:
# 일정 개수(n) 이하 작가, 출판사, 언어, 카테고리 변수 모두 Others 취급

def make_others(_column):
    tem = pd.DataFrame(books[_column].value_counts()).reset_index()
    tem.columns = ['names','count']
    others_list = tem[tem['count'] < n]['names'].values  # n은 초기에 설정함. 바꿀 수 있음.
    books.loc[books[books[_column].isin(others_list)].index, _column]= 'others'

make_others('book_author')
make_others('publisher')
make_others('language')
make_others('category_high')

In [8]:
# users 테이블 전처리 입니다.
# location이 지역, 주, 국가로 되어있어 이 부분 기초 전처리 진행 과정입니다.

users['location'] = users['location'].str.replace(r'[^0-9a-zA-Z:,]', '') # 특수문자 제거

# 지역, 주, 국가
users['location_city'] = users['location'].apply(lambda x: x.split(',')[0].strip())
users['location_state'] = users['location'].apply(lambda x: x.split(',')[1].strip())
users['location_country'] = users['location'].apply(lambda x: x.split(',')[2].strip())

users = users.replace('na', np.nan) #특수문자 제거로 n/a가 na로 바뀌게 되었습니다. 따라서 이를 컴퓨터가 인식할 수 있는 결측값으로 변환합니다.
users = users.replace('', np.nan) # 일부 경우 , , ,으로 입력된 경우가 있었으므로 이런 경우에도 결측값으로 변환합니다.

# 도시는 존재하는데 나라 정보가 없는 경우 채워주는 코드
modify_location = users[(users['location_country'].isna())&(users['location_city'].notnull())]['location_city'].values
location = users[(users['location'].str.contains('seattle'))&(users['location_country'].notnull())]['location'].value_counts().index[0]

location_list = []
for location in modify_location:
    try:
        right_location = users[(users['location'].str.contains(location))&(users['location_country'].notnull())]['location'].value_counts().index[0]
        location_list.append(right_location)
    except:
        pass

for location in location_list:
    users.loc[users[users['location_city']==location.split(',')[0]].index,'location_state'] = location.split(',')[1]
    users.loc[users[users['location_city']==location.split(',')[0]].index,'location_country'] = location.split(',')[2]

In [9]:
# 저는 도시, 주, 국가 중 주를 선택했습니다.
# 우선 모든 변수를 다 쓰는 건 아니라고 생각했어요. 도시 < 주 < 국가로 포함관계가 있기 때문이죠.
# 데이터 분석 결과 주와 국가 단위가 비슷한 경우가 많은 것을 확인했습니다.
# 조그만 섬 국가 < 미국 켈리포니아 주 < 미국 같은 경우죠.
# 미국 같은 경우 미국으로 뭉뚱그리기 보다 주 단위로 나누는 것이 맞다고 판단했습니다.
# 실제 미국 주 별로 rating 차이가 꽤 존재합니다.
# 다만 도시 기준으로 나누면 너무 세분화 될 것 같다는 생각이 들었습니다.
# 결론적으로 주를 지역을 나타내는 변수로 사용하기로 하고 결측값을 도시, 나라에서 채우기로 했습니다.

def _fillna(x):
    if pd.isna(x['location_country']):
        # 만약 나라가 기록 안되있는 경우        
        if pd.isna(x['location_city']):
            # 도시까지 없다면 모든 정보가 없음. 최빈값 california 사용.
            return 'california'
        else:
            tem = users['location_state'][users['location_city'] == x['location_city']].value_counts()
            if len(tem) == 0: 
                # 만약 주 이름이 없는 도시이면 도시 이름을 주 이름으로 사용.
                return x['location_city'] 
            else:
                # 그 도시에서 가장 자주 쓰이는 주 이름 사용.
                return tem.index[0]

    else:
        tem = users['location_state'][users['location_country'] == x['location_country']].value_counts()
        if len(tem) == 0: 
            # 만약 주 이름이 없는 나라이면 나라이름을 주 이름으로 사용.
            return x['location_country'] 
        else:
            # 그 나라에서 가장 자주 쓰이는 주 이름 사용.
            return tem.index[0]

users['fix_location_state'] = users.apply(lambda x : _fillna(x) if pd.isna(x['location_state']) else x['location_state'], axis = 1)

![image](https://user-images.githubusercontent.com/79916736/197685190-c4b88b82-4a23-4b79-b7a4-1fb849032dd3.png)

In [10]:
# 위 데이터를 바탕으로 나이를 분류합니다. 
# 이때 10세 미만과 나이가 NULL인 데이터는 상당히 유사한 평점을 메기는 것 같아요.
# 그러므로 10세 미만 사용자와 NULL 사용자는 같은 그룹으로 묶습니다. 

users['fix_age'] = users['age'].copy()
users['fix_age'][users['age'] < 10] = 10
users['fix_age'][(users['age'] < 20) & (users['age'] >= 10)] = 20
users['fix_age'][(users['age'] < 30) & (users['age'] >= 20)] = 30
users['fix_age'][(users['age'] < 35) & (users['age'] >= 30)] = 35
users['fix_age'][(users['age'] < 40) & (users['age'] >= 35)] = 40
users['fix_age'][(users['age'] < 50) & (users['age'] >= 40)] = 50
users['fix_age'][users['age'] >= 50] = 100
users['fix_age'].fillna(10, inplace = True)
users['fix_age'] = users['fix_age'].astype('int')
users['fix_age'] = users['fix_age'].astype('str')

In [11]:
# users에서 사용한 변수는 모두 제거합니다.
users = users[['user_id', 'fix_location_state', 'fix_age']]
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68092 entries, 0 to 68091
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             68092 non-null  int64 
 1   fix_location_state  68092 non-null  object
 2   fix_age             68092 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.6+ MB


In [12]:
# 일정 개수(n) 이하 fix_location_state 범주 모두 Others 취급

def make_others2(_column):
    tem = pd.DataFrame(users[_column].value_counts()).reset_index()
    tem.columns = ['names','count']
    others_list = tem[tem['count'] < n]['names'].values  # n은 초기에 설정함. 바꿀 수 있음.
    users.loc[users[users[_column].isin(others_list)].index, _column]= 'others'

make_others2('fix_location_state')

In [13]:
# n값과 함께 csv 파일로 새로 저장함

books.to_csv(f"../data/ksy_books_{n}n.csv", index = False)
users.to_csv(f"../data/ksy_users_{n}n.csv", index = False)

In [23]:
# 전처리 완료한 books와 users 테이블을 이용해 rating 테이블과 merge 하기.

train_rating = pd.merge(train,books, how='right',on='isbn')
train_rating.dropna(subset=['rating'], inplace = True)
train_rating = pd.merge(train_rating, users, how='right',on='user_id')
train_rating.dropna(subset=['rating'], inplace = True)

test['index'] = test.index
test_rating = pd.merge(test,books, how='right',on='isbn')
test_rating.dropna(subset=['rating'], inplace = True)
test_rating = pd.merge(test_rating, users, how='right',on='user_id')
test_rating.dropna(subset=['rating'], inplace = True)
test_rating = test_rating.sort_values('index')
test_rating.drop(['index'], axis=1, inplace=True)


In [24]:
train_rating.to_csv(f"../data/ksy_train_rating_{n}n.csv", index = False)
test_rating.to_csv(f"../data/ksy_test_rating_{n}n.csv", index = False)

In [22]:
test_rating = pd.merge(test,books, how='right',on='isbn')
test_rating.dropna(subset=['rating'], inplace = True)
test_rating = pd.merge(test_rating, users, how='right',on='user_id')
test_rating.dropna(subset=['rating'], inplace = True)
test_rating = test_rating.sort_values('index')
test_rating

,user_id,isbn,rating,index,book_author,publisher,language,category_high,years,fix_location_state,fix_age
6,11676.0,0002005018,0.0,0.0,others,Flamingo,en,actresses,2020,california,10
1463,116866.0,0002005018,0.0,1.0,others,Flamingo,en,actresses,2020,ontario,10
1597,152827.0,0060973129,0.0,2.0,others,Perennial,en,others,2000,ontario,50
1657,157969.0,0374157065,0.0,3.0,others,Farrar Straus Giroux,en,medical,2000,colorado,35
1912,67958.0,0399135782,0.0,4.0,Amy Tan,Putnam Pub Group,en,fiction,2000,idaho,40
...,...,...,...,...,...,...,...,...,...,...,...
54967,278543.0,1576734218,0.0,76694.0,others,Multnomah,en,family,2000,california,40
92535,278563.0,3492223710,0.0,76695.0,others,Piper,de,others,2000,wien,40
42352,278633.0,1896095186,0.0,76696.0,others,others,en,fiction,2020,utah,10
100165,278668.0,8408044079,0.0,76697.0,others,Planeta,en,fiction,2020,madrid,50
